In [ ]:
SELECT 2 + 2;

Write a query that joins the HumanResources.Employee table to the Person.Person table so that you can display the FirstName, LastName, and HireDate columns for each employee. Display the JobTitle along with a count of employees for the title

In [ ]:
SELECT FirstName, LastName, HireDate, JobTitle, 
       COUNT(*) OVER (PARTITION BY JobTitle) [Employees on this job title]
FROM HumanResources.Employee AS e
JOIN Person.Person AS p
ON e.BusinessEntityID = p.BusinessEntityID

Display the CustomerID, SalesOrderID, and OrderDate for each Sales.SalesOrderHeader row as long as the customer has placed at least five orders.

_Hint: use IN operator and subquery (query1) and use JOIN and subquery (query2)_

In [ ]:
SELECT CustomerID, SalesOrderID, OrderDate
FROM Sales.SalesOrderHeader
WHERE CustomerID IN (
    SELECT CustomerID
    FROM Sales.SalesOrderHeader
    GROUP BY CustomerID
    HAVING COUNT(*) > 4
)

In [ ]:
WITH sub AS (
    SELECT CustomerID
    FROM Sales.SalesOrderHeader
    GROUP BY CustomerID
    HAVING COUNT(*) > 4)
SELECT sub.CustomerID, SalesOrderID, OrderDate
FROM Sales.SalesOrderHeader AS s
JOIN sub ON s.CustomerID = sub.CustomerID

Write a batch that declares an integer variable called @Count to save the count of all the Sales.SalesOrderDetail records. Add an IF block that that prints “Over 100,000” if the value exceeds 100,000. Otherwise, print “100,000 or less.” (use PRINT function)


In [ ]:
DECLARE @Count INT
SET @Count = (SELECT COUNT(*) FROM Sales.SalesOrderHeader)

IF @Count > 100000
    PRINT 'Over 100,000'
ELSE
    PRINT '100,000 or less'

SELECT @Count AS c

Write a batch that contains nested IF blocks. The outer block should check to see whether the month is October or November. If that is the case, print “The month is ” and the month name. The inner block should check to see whether the year is even or odd and print the result.


In [ ]:
DECLARE @Month VARCHAR(255)
SET @Month = DATENAME(MONTH, GETDATE())
DECLARE @Year INT
SET @Year = YEAR(GETDATE())

IF @Month not in ('October', 'November')
BEGIN
    PRINT 'The month is ' +  @Month
    IF @Year % 2 = 0
        PRINT 'Even'
    ELSE
        PRINT 'Odd'
END
GO

Write a script that contains a WHILE loop that counts up from 1 to 100. Print “Odd” or “Even” depending on the value of the counter


In [ ]:
DECLARE @i AS INT = 1;
WHILE @i <= 100
BEGIN
    IF @i % 2 = 0
        PRINT 'Even'
    ELSE
        PRINT 'Odd'
    SET @i = @i + 1
END

Write a statement that attempts to insert a duplicate row into the HumanResources.Department table. Use the @@ERROR function to display the error.

Change the code you wrote in question 6 to use TRY…CATCH. Display the error number, message, and severity

Change the code you wrote in question 7 to raise a custom error message instead of the actual error message.

In [ ]:
BEGIN TRY
INSERT INTO HumanResources.Department (Name, GroupName, ModifiedDate)
SELECT Name, GroupName, ModifiedDate FROM HumanResources.Department WHERE DepartmentID = 1; 
END TRY
BEGIN CATCH
    PRINT @@ERROR;
    PRINT ERROR_MESSAGE()
    PRINT ERROR_SEVERITY()
    RAISERROR ('Custom error message', 18, 1)
END CATCH;

Create a temp table called #CustomerInfo that contains CustomerID, FirstName, and LastName columns. Include CountOfSales and SumOfTotalDue columns. Populate the table with a query using the Sales.Customer, Person.Person, and Sales.SalesOrderHeader tables

In [ ]:
CREATE TABLE dbo.#CustomerInfo (
    CustomerID INT NOT NULL IDENTITY(1, 1) PRIMARY KEY,
    FirstName VARCHAR(255),
    LastName VARCHAR(255),
    CountOfSales INT,
    SumOfTotalDue FLOAT
)

INSERT INTO dbo.#CustomerInfo (FirstName, LastName, CountOfSales, SumOfTotalDue) (
    SELECT FirstName, LastName,
    COUNT(SalesOrderID) OVER (PARTITION BY s.CustomerID),
    sum(TotalDue) OVER (PARTITION BY s.CustomerID)
    FROM Sales.Customer AS c
    JOIN Person.Person AS p
    ON c.CustomerID = p.BusinessEntityID
    JOIN Sales.SalesOrderHeader AS s
    ON c.CustomerID = s.CustomerID
)

SELECT * FROM dbo.#CustomerInfo;

In [ ]:
DROP TABLE dbo.#CustomerInfo;

Change the code written in question 9 to use a table variable instead of a temp table


In [ ]:
DECLARE @CustomerInfo TABLE (
    CustomerID INT NOT NULL IDENTITY(1, 1) PRIMARY KEY,
    FirstName VARCHAR(255),
    LastName VARCHAR(255),
    CountOfSales INT,
    SumOfTotalDue FLOAT
)

INSERT INTO @CustomerInfo (FirstName, LastName, CountOfSales, SumOfTotalDue) (
    SELECT FirstName, LastName,
    COUNT(SalesOrderID) OVER (PARTITION BY s.CustomerID),
    sum(TotalDue) OVER (PARTITION BY s.CustomerID)
    FROM Sales.Customer AS c
    JOIN Person.Person AS p
    ON c.CustomerID = p.BusinessEntityID
    JOIN Sales.SalesOrderHeader AS s
    ON c.CustomerID = s.CustomerID
)

SELECT * FROM @CustomerInfo;

Create a table variable with two integer columns, one of them an INDENTITY column. Use a WHILE loop to populate the table with 1,000 random integers using the following formula. Use a second WHILE loop to print the values from the table variable one by one.
CAST(RND() * 10000 AS INT) + 1


In [ ]:
CREATE TABLE dbo.#temp (
    ID INT NOT NULL IDENTITY(1, 1) PRIMARY KEY,
    Indentity INT
)

DECLARE @i AS INT = 1
WHILE @i <= 100
BEGIN
    INSERT INTO dbo.#temp VALUES (CAST(RAND() * 10000 AS INT) + 1)
    SET @i = @i + 1
END

In [ ]:
SELECT * FROM dbo.#temp
DROP TABLE dbo.#temp;

Write a script that declares an integer variable called @ID. Assign the value 70000 to the variable. Use the variable in a SELECT statement that returns all the SalesOrderID values from the Sales.SalesOrderHeader table that have a SalesOrderID greater than the value of the variable.


In [ ]:
DECLARE @ID INT = 70000

SELECT *
FROM Sales.SalesOrderHeader
WHERE SalesOrderID > @ID

 Write a script that declares two integer variables called @MaxID and @MinID. Use the variables to print the highest and lowest SalesOrderID values from the Sales.SalesOrderHeader table.
Hint: To convert max value to varchar use CONVERT(VARCHAR,@MaxID)


In [ ]:
DECLARE @MaxID INT
DECLARE @MinID INT

SET @MaxID = (SELECT max(SalesOrderID) FROM Sales.SalesOrderHeader)
SET @MinID = (SELECT min(SalesOrderID) FROM Sales.SalesOrderHeader)

PRINT 'Max: ' + CONVERT(VARCHAR, @MaxID) 
PRINT 'Min: ' + CONVERT(VARCHAR, @MinID) 